In [ ]:
pip install nltk

In [ ]:
pip install scikit-learn

In [ ]:
pip install sentencepiece

In [ ]:
import pandas as pd
import ast
import string
import re
import torch
from transformers import BertTokenizer,BertModel
import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")



df = pd.read_csv('C:\\Users\\Ashfaq Ahmed\\Desktop\\archive\\MovieSummaries\\movie_metadata.tsv', delimiter='\t', header=None)

df.columns = [
    'movie_id', 'freebase_id', 'title', 'release_date', 'box_office',
    'runtime', 'languages', 'countries', 'genres'
]

def extract_genre_list(genre_str):
    try:
        genre_dict = ast.literal_eval(genre_str)
        return list(genre_dict.values())
    except:
        return []

df['genre_list'] = df['genres'].apply(extract_genre_list)

# Step 1: Read movie_id and genre from TSV
df_tsv = pd.read_csv("C:\\Users\\Ashfaq Ahmed\\Desktop\\archive\\MovieSummaries\\movie_metadata.tsv", sep="\t", header=None)
df_tsv.columns = ['movie_id', 'freebase_id', 'title', 'release_date', 'box_office',
                  'runtime', 'languages', 'countries', 'genres']

# Convert genre dict string to a list of genre names
def extract_genre_list(genre_str):
    try:
        genre_dict = ast.literal_eval(genre_str)
        return list(genre_dict.values())
    except:
        return []

df_tsv['genre_list'] = df_tsv['genres'].apply(extract_genre_list)

# Keep only movie_id and genre_list
df_genres = df_tsv[['movie_id', 'genre_list']]

# Step 2: Read summaries from TXT file and make a dictionary
summary_dict = {}

with open("C:\\Users\\Ashfaq Ahmed\\Desktop\\archive\\MovieSummaries\\plot_summaries.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t", 1)  # split into movie_id and summary
        if len(parts) == 2:
            movie_id, summary = parts
            summary_dict[int(movie_id)] = summary  # store as int for matching

# Step 3: Match summaries to movie IDs
df_genres['summary'] = df_genres['movie_id'].apply(lambda x: summary_dict.get(x, ""))

# Step 4: Drop entries with missing summaries (optional)
df_final = df_genres[df_genres['summary'] != ""]

# Step 5: Save to CSV
df_final.to_csv("final_cleaned_data.csv", index=False)


df2 = pd.read_csv('final_cleaned_data.csv')


df2['genre_list'] = df2['genre_list'].apply(ast.literal_eval)


def remove_noise(data):
    data = re.sub(r'\s+', ' ', data)
    data = re.sub(r'<.*?>', '', data)
    data = re.sub(r'[^a-zA-Z0-9\s]', '', data)
    data = data.lower()
    data = re.sub(r'[\d]', '', data)                      # Remove digits
    data = data.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return data

df2['Clean_summaries'] = df2['summary'].apply(remove_noise)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

nltk.download("punkt")

# Initialize Python porter stemmer
wnl = WordNetLemmatizer()


def tokenize_and_lemmatize(text):
    tokens = tokenizer.tokenize(text)
    lemmatized = [wnl.lemmatize(word, pos="v") for word in tokens]  # pos="v" for verbs
    return ' '.join(lemmatized)

# Apply it to your cleaned summaries
df2['lemmatized_summary'] = df2['Clean_summaries'].apply(tokenize_and_lemmatize)

In [ ]:
x = df2['lemmatized_summary']
y = df2['genre_list']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
print(df2['lemmatized_summary'].head())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 1))

cv = CountVectorizer()
X_train_vectorized = cv.fit_transform(x_train)
X_test_vectorized = cv.transform(x_test)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_updated = mlb.fit_transform(y_train)
y_test_updated = mlb.transform(y_test)

In [ ]:
clf = OneVsRestClassifier(LogisticRegression())

# Train the model
clf.fit(X_train_vectorized, y_train_updated)

In [58]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, hamming_loss

# Vectorize test data
X_test_vectorized = cv.transform(x_test)  # Note: use transform, NOT fit_transform

# Predict
y_pred = clf.predict(X_test_vectorized)

# Convert predicted and actual genres back to labels
predicted_genres = mlb.inverse_transform(y_pred)
actual_genres = mlb.inverse_transform(y_test_updated)

# Show first 5 predictions
for i in range(5):
    print(f"\nSummary:\n{x_test.iloc[i][:200]}...")
    print("Predicted Genres:", predicted_genres[i])
    print("Actual Genres:", actual_genres[i])

# Evaluation Metrics
acc = accuracy_score(y_test_updated, y_pred)
print("Accuracy Score:", acc)
print("Hamming Loss:", hamming_loss(y_test_updated, y_pred))
print("F1 Score (micro):", f1_score(y_test_updated, y_pred, average='micro'))
print("F1 Score (macro):", f1_score(y_test_updated, y_pred, average='macro'))



Summary:
the film open in edinburgh with a narration by angus the hang ##man tell of how the corpses of those hang by himself be transport to dr robert knox to di ##sse ##ct while his rival dr alexander mon ##...
Predicted Genres: ('Crime Fiction', 'Drama')
Actual Genres: ('Comedy', 'Thriller')

Summary:
the story start as one of the robots fly into a scientists secret lair and un ##load ##s a pile of cash into a vault the robot be control completely from the scientists command center and we see that ...
Predicted Genres: ('Animation', 'Science Fiction', 'Short Film', 'Superhero movie')
Actual Genres: ('Action', 'Adventure', 'Animation', 'Family Film', 'Fantasy', 'Science Fiction', 'Short Film')

Summary:
the film main character be mo char ##a sc ##ud murphy and ce ##re ##ber ##al paul ##sy all from west belfast they have an interest in dog race which the narrator mo char ##a inform us be very importan...
Predicted Genres: ('Adventure', 'Animal Picture', "Children's", 'Comedy film', '

c:\Users\Ashfaq Ahmed\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [76]:
summary = input("Enter a movie summary:\n")

# Step 1: Preprocess if needed
cleaned_summary = remove_noise(summary)  # Apply same preprocessing used in training

Lemmatized_summary = tokenize_and_lemmatize(cleaned_summary)

# Step 2: Vectorize using trained CountVectorizer
summary_vectorized = cv.transform([Lemmatized_summary])  # Note: list input

# Step 3: Predict using trained classifier
prediction = clf.predict(summary_vectorized)

# Step 4: Inverse transform to get genre labels
predicted_genres = mlb.inverse_transform(prediction)

# Step 5: Output result
print("Predicted Genres:", predicted_genres[0])

Predicted Genres: ('Crime Fiction', 'Drama')


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import time
import sentencepiece

# Define a reusable translation function
def load_model_and_tokenizer(src_lang, tgt_lang):
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer

def translate_text(text, model, tokenizer):
    batch = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt", truncation=True, max_length=512)
    gen = model.generate(**batch)
    return tokenizer.decode(gen[0], skip_special_tokens=True)

# Load your dataset
df3 = pd.read_csv("C:\\Users\\Ashfaq Ahmed\\Desktop\\final_cleaned_data.csv")  # Replace with your filename

# Language pairs you want to translate to
lang_pairs = {
    "ar": ("en", "ar"),
    "ur": ("en", "ur"),
}

#removed_noise_summary = df3['summary'].apply(remove_noise)
#lemmatized_summary_for_translation = removed_noise_summary.apply(tokenize_and_lemmatize)

# Translate each cleaned summary
for lang_code, (src, tgt) in lang_pairs.items():
    print(f"Loading model for {src} → {tgt}")
    model, tokenizer = load_model_and_tokenizer(src, tgt)

    translations = []
    for i, text in enumerate(df2['lemmatized_summary'][:5]):
        try:
            translated = translate_text(text, model, tokenizer)
            translations.append(translated)
        except Exception as e:
            print(f"Error at row {i}: {e}")
            translations.append("")
        time.sleep(0.1)  # Optional: prevent memory overload

    df3.loc[:4, f'summary_{lang_code}'] = translations

# Save the translated data
df3.to_csv("translated_movie_data.csv", index=False)
print("Translation complete. Saved to translated_movie_data.csv.")